This notebook captures efforts to get the DAQ acquiring data in CORRELATOR_DATA mode, on the AAVS3 backup server in the Low ITF.

# Preliminary stuff to make sure everything is on:

In [1]:
import os
import json
import tango

os.environ["TANGO_HOST"] = "databaseds-tango-base-test.ska-low-mccs:10000"

In [2]:
subrack = tango.DeviceProxy("low-mccs/subrack/aavs3-1")

In [3]:
subrack.adminMode

<adminMode.OFFLINE: 1>

In [4]:
subrack.adminMode = 0

In [5]:
subrack.state()

tango._tango.DevState.ON

In [7]:
subrack.tpmPresent

array([False, False,  True, False, False, False,  True, False])

In [8]:
subrack.tpm3powerstate

<tpm3PowerState.ON: 4>

In [9]:
subrack.tpm7PowerState

<tpm7PowerState.ON: 4>

In [10]:
tile = tango.DeviceProxy("low-mccs/tile/aavs3-01")

In [11]:
tile.adminMode

<adminMode.OFFLINE: 1>

In [12]:
tile.adminMode = 0

In [13]:
tile.state()

tango._tango.DevState.ON

In [14]:
daq = tango.DeviceProxy("low-mccs/daqreceiver/aavs3")

In [15]:
daq.adminMode

<adminMode.OFFLINE: 1>

In [16]:
daq.adminMode = 0

In [17]:
daq.state()

tango._tango.DevState.ON

In [19]:
tile.tileprogrammingstate

'Initialised'

# Tell TPM to start acquiring data from the antennas

In [20]:
tile.startacquisition('{}')

[array([2], dtype=int32),
 ['1692956023.0753045_186332062180875_StartAcquisition']]

In [21]:
tile.tileprogrammingstate

'Synchronised'

# Configure the TPM's LMC data spigot

In [26]:
dst_tpm_port = 4660
dst_tpm_ip = "data-receiver-aavs3-data"
lmc_download_dict = {"mode": "10g", "destination_ip": dst_tpm_ip, "destination_port": dst_tpm_port}
tile.SetLmcDownload(json.dumps(lmc_download_dict))

[array([0], dtype=int32), ['SetLmcDownload command completed OK']]

# Configure the DAQ

In [32]:
daq_config = {
    "nof_antennas": 256,
    "nof_tiles": 1,
    "acquisition_duration": 30,
    "receiver_frame_size": 9000,
}
daq.Configure(json.dumps(daq_config))

[array([0], dtype=int32), ['Configure command completed OK']]

# Start the DAQ acquiring data in CORRELATOR_DATA mode

In [33]:
daq.Start('{"modes_to_start": "DaqModes.CORRELATOR_DATA"}')

[array([2], dtype=int32), ['1692956407.3771927_171148796922609_Start']]

In [34]:
daq.daqStatus()

'{"Running Consumers": [["CORRELATOR_DATA", 7]], "Receiver Interface": "eth0", "Receiver Ports": [4660], "Receiver IP": ["10.10.164.7"], "Daq Health": ["OK", 0]}'

# Tell the TPM to send channel data on its LMC spigot

In [36]:
tile.SendDataSamples(
    json.dumps(
        {
            "channel_id": 100,
            "data_type": "channel_continuous",
        }
    )
)

[array([0], dtype=int32), ['SendDataSamples command completed OK']]

# Weep
At this point we were expecting the DAQ to receive data; but it didn't.
We think the problem is that the TPM and DAQ are on different subnets, and we were sending data over the 1Gb link, which does not have gateway support.

# Clean up

In [31]:
daq.Stop()

[array([0], dtype=int32), ['Daq stopped']]